# 1. Problem Statement

We need to model a NN that can classify whether a given image is Building, Forest, Glacier, Mountain, Sea, or Street.

We are using CNN to train our model. (using a gpu)

# 2. Importing Libraries

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

### Checking number of gpu

In [2]:
len(tf.config.list_physical_devices('GPU'))

1

# 3. Getting dataset

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory('seg_train',
                                                      validation_split = 0.2,
                                                       subset = 'training',
                                                       seed = 123,
                                                       image_size = (150, 150),
                                                       batch_size = 100
                                                      )

Found 14034 files belonging to 6 classes.
Using 11228 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory('seg_train',
                                                      validation_split = 0.2,
                                                       subset = 'validation',
                                                       seed = 123,
                                                       image_size = (150, 150),
                                                       batch_size = 100
                                                      )

Found 14034 files belonging to 6 classes.
Using 2806 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


# 4. Modeling

In [6]:
model = tf.keras.Sequential()

model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(150, 150, 3)))
model.add(layers.Conv2D(16,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(16,kernel_size=(3,3), activation = 'relu',))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(32,kernel_size=(3,3), activation = 'relu',))
model.add(layers.Conv2D(32,kernel_size=(3,3), activation = 'relu',))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(56,kernel_size=(3,3), activation = 'relu',))
model.add(layers.Conv2D(56,kernel_size=(3,3), activation = 'relu',))
model.add(layers.Conv2D(56,kernel_size=(3,3), activation = 'relu',))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(128,kernel_size=(3,3), activation = 'relu',))
model.add(layers.Conv2D(128,kernel_size=(3,3), activation = 'relu',))
model.add(layers.Conv2D(128,kernel_size=(3,3), activation = 'relu',))
model.add(layers.MaxPool2D())

model.add(layers.Flatten())
model.add(layers.Dense(256, 'relu'))
model.add(layers.Dense(128, 'relu'))
model.add(layers.Dense(6,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 34, 32)        0

# 5. Training

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          epochs=100,
          validation_data = val_ds,
          callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
          verbose = 1)

Epoch 1/100
113/113 [==============================] - 41s 252ms/step - loss: 1.3382 - accuracy: 0.4107 - val_loss: 1.0304 - val_accuracy: 0.5606
Epoch 2/100
113/113 [==============================] - 26s 231ms/step - loss: 1.0307 - accuracy: 0.5688 - val_loss: 0.9245 - val_accuracy: 0.6194
Epoch 3/100
113/113 [==============================] - 26s 227ms/step - loss: 0.9030 - accuracy: 0.6372 - val_loss: 0.8843 - val_accuracy: 0.6475
Epoch 4/100
113/113 [==============================] - 26s 226ms/step - loss: 0.8434 - accuracy: 0.6687 - val_loss: 0.7766 - val_accuracy: 0.7067
Epoch 5/100
113/113 [==============================] - 26s 227ms/step - loss: 0.7605 - accuracy: 0.7065 - val_loss: 0.7374 - val_accuracy: 0.7270
Epoch 6/100
113/113 [==============================] - 26s 227ms/step - loss: 0.6829 - accuracy: 0.7406 - val_loss: 0.7765 - val_accuracy: 0.7167
Epoch 7/100
113/113 [==============================] - 26s 229ms/step - loss: 0.6483 - accuracy: 0.7525 - val_loss: 0.6391 -

train_loss = 0.4634

val_loss = 0.5822

As, val_loss is only slighly higher than train_loss.<br>
**The model is not overfitted.**

# 6. testing and accuracy

In [8]:
test_ds = tf.keras.utils.image_dataset_from_directory('seg_test',
                                                       image_size = (150, 150),
                                                       batch_size = 100
                                                      )

Found 3000 files belonging to 6 classes.


In [9]:
model.evaluate(test_ds, verbose=1)

30/30 [==============================] - 3s 89ms/step - loss: 0.6656 - accuracy: 0.7760


[0.665601372718811, 0.7760000228881836]

test_loss = 0.66

And the **accuracy of the model is about 78 %.**